<a href="https://colab.research.google.com/github/Svetorus/Pandas/blob/main/Read_write_and_pip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Определить тип файловой системы
import os
os.system('uname -o')

0

In [6]:
# Определить тип файловой системы
from sys import platform
if platform == "linux" or platform == "linux2":
    print(f'linux')
elif platform == "darwin":
    print(f'OS X')
elif platform == "win32":
    print(f'win32')

linux


In [7]:
# В python текущую директорию можно узнать, выполнив команды:
import os
print(os.getcwd())

/content


In [ ]:
# Поменять текущую директорию из Python можно так:
import os
os.chdir(r'\\Справочники\Терминал')

In [ ]:
# Прочитать файл из linux в папке windows 
# библиотека SAMBA
import smbclient
smbclient.ClientConfig(username='Name', password='password')
with smbclient.open_file(r"\\Справочники\Терминал\фаил.csv", mode="r", encoding='UTF-8') as fd:
    res=fd.read()
res = res.split('\n')
df =pd.DataFrame(res)
df = df[0].str.split(';',expand=True)
df.columns = df.iloc[0]
df = df.drop(0, axis=0)
df

# Чтение файлов
Рассматриваем задачу сквозной аналитики. Будем работать с этими файлами:
- visit_log.csv
- purchase_log.txt

### Способ 1. Делаем все пошагово
Опции при открытии файлов:
- 'r' открыть для чтения. Файл при этом не меняется
- 'w' открыть файл для записи. Файл при этом полностью чистится (!)
- 'a' открыть файл для добавления. Файл не меняется, все записи добавляются в конец

In [ ]:
# открываем файл для чтения (опция r)
f = open('visit_log.csv', 'r', encoding='utf-8')

In [ ]:
f

<_io.TextIOWrapper name='visit_log.csv' mode='r' encoding='utf-8'>

In [ ]:
f.__sizeof__()

192

In [ ]:
# прочитать первую строчку
f.readline()

'user_id,source\n'

In [ ]:
# прочитать еще одну
f.readline()

'96064ae9e0,other\n'

In [ ]:
# можно и так
next(f)

'6450655ae8,other\n'

In [ ]:
# прочитать все содержимое файла в переменную content
content = f.readlines()

In [ ]:
content[:5]

['71bcf169b4,other\n',
 'e0aee73c5d,other\n',
 '1f8845e157,context\n',
 '78d22c5c78,other\n',
 'dd85040770,other\n']

In [ ]:
content[-1]

'c23704ea54,other\n'

In [ ]:
f.seek(0)

0

In [ ]:
f.readline()

'user_id,source\n'

In [9]:
# построчное чтение файла
for line in f:
    print(line)
    
    # break

In [ ]:
i = 0
for line in f:
    print(line)
    
    i += 1
    if i > 5:
        break

d94b908660,None

f697206af5,other

8a9a2de36e,other

dd3d43c266,context

f6401d302f,context

b18d58560b,email



In [ ]:
for i, line in enumerate(f):
    print(i, line)
    
    if i > 5:
        break

0 ed79586589,other

1 1840e0b9d4,other

2 b4afdc5ccc,other

3 470248aabf,other

4 948d9be43c,other

5 03592d16ac,other

6 8554e1a9f9,context



Часто используется конструкция 
```python
another_line.strip().split(',')
```

In [ ]:
another_line = f.readline()
another_line

'ceda3a4603,other\n'

In [ ]:
# удаляем перенос строки и лишние пробелы
another_line.strip()

'ceda3a4603,other'

In [ ]:
';ceda3a4603,other\n;--!--'.strip(';-!')

'ceda3a4603,other\n'

In [ ]:
# разбиваем столбцы
another_line.strip().split(',')

['ceda3a4603', 'other']

In [ ]:
# закрытие файла
f.close()

In [ ]:
# после закрытия не получится прочитать
f.readline()

ValueError: I/O operation on closed file.

In [ ]:
# прочитать все строчки файла в список (т. е. в оперативную память)
f = open('visit_log.csv', 'r')

content = f.readlines()

f.close()

In [ ]:
content[:5]

['user_id,source\n',
 '6450655ae8,other\n',
 'b4ea53e670,other\n',
 '0e37347152,other\n',
 '96064ae9e0,other\n']

# Запись в файл
Зачем все-таки открывать и закрывать файлы?

In [ ]:
f = open('results.csv', 'w')

In [ ]:
f.write('Начинаю запись первой строки...\n')

32

In [ ]:
f.write('Начинаю запись второй строки...\n')

32

Здесь кто-то еще хочет в него записать

In [ ]:
my_friend_results = open('results.csv', 'w')

In [ ]:
my_friend_results.write('Тут еще результаты есть')

23

In [ ]:
# пишем свой результат
f.close()

In [ ]:
# и наш результат перезатирается
my_friend_results.close()

### Контекстный менеджер
Как обезопасить себя от подобных накладок

In [ ]:
with open('results.csv', 'w') as f:
    f.write('Начинаю запись первой строки...\n')
    f.write('Начинаю запись второй строки...\n')
    
    my_friend_results = open('results.csv', 'w')
    my_friend_results.write('Тут еще результаты есть')
    my_friend_results.close()

### А можно читать и сразу в файл записывать?
Напишите функцию, которая фильтрует файл visit_log.csv по источнику context и пишет результат в visits_context.csv. Используйте функцию из второго упражнения для проверки результата.

In [ ]:
with open('visit_log.csv', 'r') as f:
    with open('visits_context.csv', 'w') as f2write:
        for line in f:
            if 'context' in line:
                f2write.write(line)

# Чтение списков и словарей из файла
Смотрим что в файле purchase_log.txt. Похоже тут не строки, а словари

In [ ]:
with open('purchase_log.txt') as f:
    print([next(f) for x in range(5)])

['{"user_id": "user_id", "category": "category"}\n', '{"user_id": "1840e0b9d4", "category": "Продукты"}\n', '{"user_id": "4e4f90fcfb", "category": "Электроника"}\n', '{"user_id": "afea8d72fc", "category": "Электроника"}\n', '{"user_id": "373a6055fe", "category": "Бытовая техника"}\n']


In [ ]:
import json

In [ ]:
# перевод строки в словарь
dict_in_string = '{"a": 1, "b": 2}'

type(json.loads(dict_in_string))

dict

In [ ]:
json.loads(dict_in_string)

{'a': 1, 'b': 2}

In [ ]:
# перевод строки в список
list_in_string = '[1, 2, 3]'

json.loads(list_in_string)[-1]

3

In [ ]:
json.loads(list_in_string)

[1, 2, 3]

In [ ]:
i = 0
with open('purchase_log.txt') as f:
    for line in f:
        line = line.strip()
        
        dict_ = json.loads(line)
        print(dict_, type(dict_))
        
        i += 1
        if i > 5:
            break

{'user_id': 'user_id', 'category': 'category'} <class 'dict'>
{'user_id': '1840e0b9d4', 'category': 'Продукты'} <class 'dict'>
{'user_id': '4e4f90fcfb', 'category': 'Электроника'} <class 'dict'>
{'user_id': 'afea8d72fc', 'category': 'Электроника'} <class 'dict'>
{'user_id': '373a6055fe', 'category': 'Бытовая техника'} <class 'dict'>
{'user_id': '9b2ab046f3', 'category': 'Электроника'} <class 'dict'>


In [ ]:
dict_ = 1

### Из словаря в строку тоже можно

In [ ]:
data = {'user_id': '1840e0b9d4', 'category': 'Продукты'}

In [ ]:
str(data)

"{'user_id': '1840e0b9d4', 'category': 'Продукты'}"

In [ ]:
json.dumps(data)

'{"user_id": "1840e0b9d4", "category": "\\u041f\\u0440\\u043e\\u0434\\u0443\\u043a\\u0442\\u044b"}'

In [ ]:
type(json.dumps(data))

str

# Модуль pickle
Запись объекта сразу в файл как поток байтов

In [ ]:
import pickle

In [ ]:
data = {'user_id': '1840e0b9d4', 'category': 'Продукты'}

In [ ]:
with open('data.pickle', 'wb') as f:
    pickle.dump(data, f)

In [ ]:
# прочитать объект из такого файла

with open('data.pickle', 'rb') as f:
    dict_ = pickle.load(f)
    
dict_, dict_['user_id']

({'user_id': '1840e0b9d4', 'category': 'Продукты'}, '1840e0b9d4')

# Чтение файлов из папки

In [ ]:
import os

In [ ]:
# чтение файлов и папок
for file in os.listdir('data'):
    if '.csv' in file:
        print(file)

file_2.csv
file_3.csv
file_1.csv


In [ ]:
# чтение всех файлов и папок, в том числе вложенных
for root, directory, file in os.walk('data'):
    print(root, directory, file)

data ['more_data'] ['.DS_Store', 'file_2.csv', 'file_3.csv', 'file_1.csv']
data/more_data [] ['file_4.csv', 'file_5.csv']


# Каталог пакетов pip (Python Package Index)
Варианты установки
1. С помощью Anaconda Navigator
2. В командной строке (Terminal на маке)
```bash
pip install package_name
```
3. Скачать версию с github и установить вручную
```bash
pip install .
```

Пакеты устанавливаются в определенное окружение! Вам пригодится virtualenv

In [ ]:
!pip install pyyaml

In [ ]:
import plotly